In [4]:
import numpy as np
import json
import cv2
import os
from math import ceil, floor

In [5]:
%cd C:/Users\na062/Desktop/sw_pro/video_dataset

C:\Users\na062\Desktop\sw_pro\video_dataset


### 비디오 불러오기 

|
|- 유동 인구 분석을 위한 CCTV 영상 데이터 / image
                 |                            |
                                            ------
                                            |     |
                                          train   Validation

In [6]:
#Json 데이터에서 box,Frame, id 꺼내오기 
def InFoOfVideo(jsonData):
    Id= []
    Box=[]
    Frame=[]
    for  annotation in jsonData['annotations']:
        if  annotation['id'] not in Id:
            Box.append(annotation['bbox'])
            Frame.append(annotation['frame'])
            Id.append(annotation['id'])
    return Box,Frame,Id

In [7]:
def Int(Box): # 조금 큰 이미지로 자르기
    Box[0] = floor(Box[0])
    Box[2] = floor(Box[2])
    Box[1] = ceil(Box[1])
    Box[3] = ceil(Box[3])
    return Box

In [8]:
def ImagesInVideo(Cap,Boxes,Frames,num,size): # cap=videocapture로 가져와 video에서 캡쳐하기 
    images=[]
    for i in range(num):
        frame = Frames[i]
        box = Int(Boxes[i])
        Cap.set(cv2.CAP_PROP_POS_FRAMES,frame) # 해당 프레임에 접근
        ret,image = Cap.read()
        
        images.append(image[box[1]:box[3],box[0]:box[2]]) if ret and 0 not in list(image.shape) else 0
    return images

In [9]:
def imwrite(filename, img, params=None):
    result, n = cv2.imencode('.jpg', img)
    if result:
        with open(filename, 'wb') as f:
            n.tofile(f)

In [21]:
def MakingImagePerVideo(videopath,labelpath,Count,train=True):
    trainpath = './image/Train'
    validpath = './image/Validation'
    count=0
    Path = trainpath if train else validpath # Train, valid 구분하기
    cap = cv2.VideoCapture(videopath) # 비디오 파일 접근 
    with open(labelpath, 'r',encoding = 'utf-8' or 'cp949' ) as f: # json 파일 접근 
        json_data = json.load(f)
        
    boxes,frames,ids = InFoOfVideo(json_data) # 정보 얻기
    Size = json_data['video']['resolution'][:-1] # 비디오 데이터 크기 얻기
    
    ImageList = ImagesInVideo(cap,boxes,frames,len(ids),Size) #이미지 추출 
    
    # 사진 저장하기
    videoname = videopath.split('/')[-1][:-4] # 비디오 이름 추출 
    
    for i in range(len(ImageList)):
        imagename = videoname + '_'+ids[i]
        path = Path+'/'+imagename+ '.jpg'
        try:
            imwrite(path,ImageList[i]) if os.path.isfile(path)==False else 0
        except:
            print("{} 비디오 {}번째 사진에서 오류 발생".format(videoname,i+1))
        else:
            count+=1
    return count

In [22]:
StdPath = './유동 인구 분석을 위한 CCTV 영상 데이터'
Train=True
for typefolder in os.listdir(StdPath):
    temppath = StdPath+'/'+typefolder # Train,validation으로 접근
    Train = True if typefolder=='Training' else False
    temp_label = temppath+'/Label' # 라벨링 데이터 모음 폴더으로 접근 
    temp_video = temppath+'/Video' # 비디오 데이터 모읍 파일 폴더로 접근
    for place_folder in os.listdir(temp_video): # 장소에 대한 폴더로 접근
        count=0
        for time_folder in os.listdir(temp_video +'/'+ place_folder): # 시간 폴더에 대한 접근 
            time = 0
            for file in os.listdir(temp_video +'/'+place_folder + '/'+time_folder): # 각각 비디오 파일 이름에 대한 정보  
                VideoPath = temp_video +'/'+ place_folder + '/'+time_folder+'/'+file
                LabelPath = temp_label +'/'+ place_folder + '/'+time_folder+'/'+ file[:-4]+'.json' 
                num = MakingImagePerVideo(VideoPath,LabelPath,count,Train) # 작업 시작 
                #print("{} 비디오 추출 완료 몇장: {} \n".format(file[:-4],num))
                time+=num
                count+=num
            print("{} 시간 폴더에서 {}장 생성".format(time_folder,time))
            print()
        print("{} 해당 장소 이미지 개수: {} ".format(place_folder,count))
        print()
    break

2021-08-04_10-06-00_wed_sunny_out_ju-ja_C0241 비디오 1번째 사진에서 오류 발생
오전(9~12) 시간 폴더에서 2407장 생성

3_ju-ja_1 해당 장소 이미지 개수: 2407 

오후(12~16) 시간 폴더에서 2992장 생성

저녁(16~20) 시간 폴더에서 3556장 생성

4_ju-ja_2 해당 장소 이미지 개수: 6548 

새벽(0~9) 시간 폴더에서 2046장 생성

야간(20~24) 시간 폴더에서 2381장 생성

저녁(16~20) 시간 폴더에서 3442장 생성

5_sa-bt_1 해당 장소 이미지 개수: 7869 

오전(9~12) 시간 폴더에서 2927장 생성

오후(12~16) 시간 폴더에서 2369장 생성

6_sa-bt_2 해당 장소 이미지 개수: 5296 

새벽(0~9) 시간 폴더에서 1678장 생성

저녁(16~20) 시간 폴더에서 3463장 생성

7_sa-st_1 해당 장소 이미지 개수: 5141 

야간(20~24) 시간 폴더에서 1882장 생성

오후(12~16) 시간 폴더에서 2472장 생성

8_sa-st_2 해당 장소 이미지 개수: 4354 

오전(9~12) 시간 폴더에서 2228장 생성

9_sa-st_3 해당 장소 이미지 개수: 2228 



2021-08-04_10-06-00_wed_sunny_out_ju-ja_C0241 비디오 1번째 사진에서 오류 발생